In [ ]:
#| hide
from dynamic_cfg.core import *

# dynamic_cfg

> Turning Classifier-free Guidance (CFG) into a dynamic process.  


## Introduction  


In Diffusion models, the standard Classifier-free Guidance (CFG) uses a scalar value, typically called `guidance_scale`, to steer the diffusion process. The guidance update is based on two latents: one from the given text prompt and another from an empty (or unconditioned) input. CFG then updates the prediction by mixing these two latents as follows:    

$$\text{Prediction} = \text{Unconditioned} + \textbf{guideScale} \cdot \left(\text{Conditioned} - \text{Unconditioned}\right) $$

We should mention that in Stable Diffusion `v2*` models, negative prompts have become very important. Now, instead of using an empty, unconditioned input we pass a negative prompt that explicitly tells the generation what to avoid. In this case, the Unconditioned latent becomes a Negative latent. The diffusion process then steers the generated image away from the Negative vector.  

We can compare Negative prompts to the famous word-vector example of `Queen = King - Man`. It's as if our text prompt was `King`, our negative prompt was `Man`, and we want to steer away the diffusion process away until it reaches `Queen`.  

## Why should Guidance be dynamic?  


The Guidance process described above currently produces incredible output images. However, there is an open question: should `guidance_scale` be a constant? This question is motivated by how different the initial and final images are from each other. Diffusions starts with random, formless noise and ends up with a complex image. What if `guidance_scale` behaved more like `sigma` (the noise schedule) and changed throughout the process? Could there even be an optimal, inverse relationship between `sigma` and `guidance_scale`? 

For example, maybe `guidance_scale` should be high early on to steer the image away from random noise. But, as the image starts taking shape, a lower `guidance_scale` could allow the model to "hallucinate" more interesting details. 

The introduction of Negative prompts also feeds into the question. We've gone from steering the output away from a neutral, unconditioned input to now steering it away from an explicitly "bad" region. How does this affect the diffusion process? Do we want to get away from the negative region even faster than before?  


More details on making CFG dynamic are available in the official [blog post]() for this library.



## dynamic Classifier-free Guidance (dCFG)

This library provides tools to explore and hopefully answer the question: how should `guidance_scale` change throughout diffusion?  

We introduce a `DynamicCFG` class to dynamically change the Guidance in two ways:  

1. Schedules
2. Normalizations

### Schedules  

Scheduling the `guidance_scale` is inspired by the tremendous success of scheduling neural network hyperparameters such as:  
- the learning rate
- optimizer momentum
- batch size  

In fact scheduled learning rates are the default in many of the most powerful modern machine learning approaches. 

Diffusion models are likewise trained with a noise schedule for `sigma`. That means that each step in the forward noisy process gets its own, time-dependent `sigma`. During CFG, we are trying to undo this noisy process to carve an image out of pure noise. Specifically, we are trying to generate an image based on the input text prompt. It is reasonable to think that our carving at each step should be tailored to the current noise level.  

To that end, we introduce a `GuidanceSchedule` class that implements the following types of schedules:  

- Constant  
- Linear  
- Cosine  

`Constant` represents that fixed, static baseline currently used by most CFG approaches. For `Linear` and `Cosine` schedules, every Guidance update gets its own, time-indexed `guidance_scale` based on the chosen schedule parameters.

### Normalizations  

Normalizing the guidance update has a few motivations. To start, a large `guidance_scale` places the vectors on very different scales. It might be better for the update vector to scale the latents or predictions.  

Likewise, normalizing the guidance update has been crucial for [Text-to-Speech approaches with diffusion](https://arxiv.org/pdf/2205.15370.pdf). Specifically, normalizing the TTS guidance update makes the generated speech actually understandable. It is reasonable to think we could also benefit from generating "intelligible" images via normalizations.  



:::: {.callout-note}  

Normalizations work extremely well in `v1*` Stable Diffusion models, and the `base v2` models. However, the new `v-prediction` objective for the `768-v` models does not work with normalizations. We likely need to rethink where and how to apply normalizations to better align with `v-prediction`.  

::::

## Install

```sh
pip install dynamic_cfg
```

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2